In [ ]:
!pip install tensorflow==2.6.0
import tensorflow as tf
print(tf.__version__)

In [ ]:
!pip install tensorflow==2.6.0 keras==2.6.0

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet101
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import metrics

# 데이터 경로
train_dir = '/content/drive/MyDrive/data2_class/train'
val_dir = '/content/drive/MyDrive/data2_class/val'

# 이미지 크기 및 채널
img_size = (224, 224)
img_channel = 3

# 이미지 데이터 전처리
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    brightness_range=(0.8,1.8),
    fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical') # categorical, binary, sparse 선택

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical') # categorical, binary, sparse 선택

# ResNet101 모델 불러오기
resnet = ResNet101(input_shape=(img_size[0], img_size[1], img_channel), weights='imagenet', include_top=False) # ImageNet으로 사전 학습된 가중치를 초기 가중치로 사용

# 새로운 분류층 추가
x = resnet.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, activation='relu')(x)
predictions = layers.Dense(10, activation='softmax')(x) # 클래스 개수만큼 조정

model = models.Model(inputs=resnet.input, outputs=predictions)

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', metrics.Precision(), metrics.Recall()])

# 체크포인트를 저장할 경로
checkpoint_filepath = '/content/drive/MyDrive/checkpoint/checkpoints/model-{epoch:03d}.h5'

# 체크포인트 콜백 설정
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_loss',
    save_best_only=True, # 가장 성능이 좋은 가중치 저장 여부
    save_weights_only=True, # 모델, 가중치 저장 선택
    verbose=1)

# 조기 종료 콜백 설정
#early_stopping_callback = EarlyStopping(
#    monitor='val_loss',
#    patience=5, # val_loss가 최소 5 epoch 동안 개선되지 않으면 학습 중단
#    restore_best_weights=True) # 가장 성능이 좋은 가중치 복원 여부

# 모델 학습
history = model.fit(train_generator,
          steps_per_epoch=len(train_generator),
          epochs=30,
          validation_data=validation_generator,
          validation_steps=len(validation_generator),
          callbacks=[checkpoint_callback]) # 체크포인트, 조기 종료 인자 추가

Found 6995 images belonging to 10 classes.
Found 1994 images belonging to 10 classes.
171446536/171446536 [==============================] - 7s 0us/step
Epoch 1/30
438/438 [==============================] - ETA: 0s - loss: 0.4839 - accuracy: 0.8562 - precision: 0.9209 - recall: 0.8176
Epoch 1: val_loss improved from inf to 2.32529, saving model to /content/drive/MyDrive/checkpoint/checkpoints/model-001.h5
438/438 [==============================] - 4711s 11s/step - loss: 0.4839 - accuracy: 0.8562 - precision: 0.9209 - recall: 0.8176 - val_loss: 2.3253 - val_accuracy: 0.1274 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/30
438/438 [==============================] - ETA: 0s - loss: 0.0907 - accuracy: 0.9720 - precision: 0.9770 - recall: 0.9700
Epoch 2: val_loss did not improve from 2.32529
438/438 [==============================] - 325s 742ms/step - loss: 0.0907 - accuracy: 0.9720 - precision: 0.9770 - recall: 0.9700 - val_loss: 9.3168 - val_accuracy: 0.0622 - val_precision

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet101
from tensorflow.keras import layers, models
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

train_dir = '/content/drive/MyDrive/data2_class/train'
val_dir = '/content/drive/MyDrive/data2_class/val'

img_size = (224, 224)
img_channel = 3

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    brightness_range=(0.8,1.8),
    fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical') # categorical, binary, sparse 선택

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical') # categorical, binary, sparse 선택

resnet = ResNet101(input_shape=(img_size[0], img_size[1], img_channel), weights=None, include_top=False) # 가중치 체크포인트 파일을 불러오므로 None으로 설정

x = resnet.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, activation='relu')(x)
predictions = layers.Dense(10, activation='softmax')(x) # 클래스 개수만큼 조정

model = models.Model(inputs=resnet.input, outputs=predictions)

# 이전에 저장된 모델 가중치 불러오기
model.load_weights('/content/drive/MyDrive/checkpoint/checkpoints/model-015.h5') # 경로 작성

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', metrics.Precision(), metrics.Recall()])

checkpoint_filepath = '/content/drive/MyDrive/checkpoint/checkpoints/model-{epoch:03d}.h5'

checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=True,
    verbose=1)

#early_stopping_callback = EarlyStopping(
#    monitor='val_loss',
#    patience=5,
#    restore_best_weights=True)

# 학습 이어서 시작
history = model.fit(train_generator,
          steps_per_epoch=len(train_generator),
          epochs=50,
          validation_data=validation_generator,
          validation_steps=len(validation_generator),
          callbacks=[checkpoint_callback],
          initial_epoch=15) # initial_epoch = 이전에 중단된 epoch 숫자

Found 6995 images belonging to 10 classes.
Found 1994 images belonging to 10 classes.
Epoch 16/50
438/438 [==============================] - ETA: 0s - loss: 0.0316 - accuracy: 0.9936 - precision: 0.9939 - recall: 0.9934 
Epoch 16: val_loss improved from inf to 0.00106, saving model to /content/drive/MyDrive/checkpoint/checkpoints/model-016.h5
438/438 [==============================] - 6508s 15s/step - loss: 0.0316 - accuracy: 0.9936 - precision: 0.9939 - recall: 0.9934 - val_loss: 0.0011 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 0.9990
Epoch 17/50
438/438 [==============================] - ETA: 0s - loss: 0.0461 - accuracy: 0.9898 - precision: 0.9900 - recall: 0.9898
Epoch 17: val_loss did not improve from 0.00106
438/438 [==============================] - 335s 765ms/step - loss: 0.0461 - accuracy: 0.9898 - precision: 0.9900 - recall: 0.9898 - val_loss: 0.1504 - val_accuracy: 0.9569 - val_precision: 0.9578 - val_recall: 0.9564
Epoch 18/50
438/438 [===================

KeyboardInterrupt: ignored

In [2]:
import matplotlib.pyplot as plt
import numpy as np

# 학습 곡선 그리기
num_epochs = len(history.history['accuracy'])
epoch_range = np.arange(1, num_epochs + 1)

plt.figure(figsize=(10, 5))
plt.plot(epoch_range, history.history['accuracy'])
plt.plot(epoch_range, history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.figure(figsize=(10, 5))
plt.plot(epoch_range, history.history['loss'])
plt.plot(epoch_range, history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.figure(figsize=(10, 5))
plt.plot(epoch_range, history.history['precision'])
plt.plot(epoch_range, history.history['val_precision'])
plt.title('Model Precision')
plt.ylabel('Precision')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.figure(figsize=(10, 5))
plt.plot(epoch_range, history.history['recall'])
plt.plot(epoch_range, history.history['val_recall'])
plt.title('Model Recall')
plt.ylabel('Recall')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

NameError: ignored

In [4]:
# 구글 드라이브에 모델 저장
model.save('/content/drive/MyDrive/ResNet101_New_Data_10Class.h5')

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import matplotlib.pyplot as plt

# 테스트 데이터 폴더 경로
test_dir = '/content/drive/MyDrive/dataset/test'

image_size = (224, 224)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=1,
    shuffle=False,
    class_mode='categorical')

# 저장된 모델 경로
model = load_model('/content/drive/MyDrive/ResNet101_New_Data_10Class.h5')

# 분류 및 평가
Y_pred = model.predict(test_generator, steps=len(test_generator))
y_pred = np.argmax(Y_pred, axis=1)

true_labels = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# 출력
print('테스트 결과')
print('---------------------')
for i, filename in enumerate(test_generator.filenames):
    print('{} : {}'.format(filename, class_labels[y_pred[i]]))

accuracy = accuracy_score(true_labels, y_pred)
precision = precision_score(true_labels, y_pred, average='macro')
recall = recall_score(true_labels, y_pred, average='macro')
f1score = f1_score(true_labels, y_pred, average='macro')

print('---------------------')
print('정확도 : {:.2f}'.format(accuracy))
print('정밀도 : {:.2f}'.format(precision))
print('재현율 : {:.2f}'.format(recall))
print('F1-score : {:.2f}'.format(f1score))

Found 1301 images belonging to 16 classes.
1301/1301 [==============================] - 955s 733ms/step
테스트 결과
---------------------
back_left/back_left103.jpg : back_left_left
back_left/back_left120.jpg : back_left_hurray
back_left/back_left121.jpg : back_left_hurray
back_left/back_left122.jpg : back_left_hurray
back_left/back_left145.jpg : front_hurray
back_left/back_left158.jpg : back_right_right
back_left/back_left182.jpg : front_hurray
back_left/back_left184.jpg : front_hurray
back_left/back_left186.jpg : front_hurray
back_left/back_left191.jpg : front_hurray
back_left/back_left21.jpg : back_right_right
back_left/back_left214.jpg : front_hurray
back_left/back_left234.jpg : front_hurray
back_left/back_left236.jpg : front_hurray
back_left/back_left241.jpg : front_hurray
back_left/back_left250.jpg : front_hurray
back_left/back_left261.jpg : back_left_hurray
back_left/back_left283.jpg : back_left_hurray
back_left/back_left294.jpg : back_left_hurray
back_left/back_left296.jpg : back_le

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
